In [133]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
db=pd.read_csv("coffee-prices-historical-data.csv")
db.dropna(inplace=True)
db["date"]=pd.to_datetime(db["date"])
db.rename(columns={" value":"target"},inplace=True)
db["prima1"]=db["target"].shift(1)
db["prima2"]=db["target"].shift(2)
db.dropna(inplace=True)
# scalerprima=StandardScaler()
# db[["prima1","prima2"]]=scalerprima.fit_transform(db[["prima1","prima2"]])
# scalertarget=StandardScaler()
# db["target"]=scalertarget.fit_transform(db[["target"]])
db.set_index('date', inplace=True)


print(db.dtypes)
print(db.tail(20))

target    float64
prima1    float64
prima2    float64
dtype: object
            target  prima1  prima2
date                              
2023-07-10  1.5985  1.5937  1.6011
2023-07-11  1.5710  1.5985  1.5937
2023-07-12  1.5711  1.5710  1.5985
2023-07-13  1.5795  1.5711  1.5710
2023-07-14  1.5918  1.5795  1.5711
2023-07-17  1.5483  1.5918  1.5795
2023-07-18  1.5563  1.5483  1.5918
2023-07-19  1.5567  1.5563  1.5483
2023-07-20  1.5849  1.5567  1.5563
2023-07-21  1.6053  1.5849  1.5567
2023-07-24  1.6306  1.6053  1.5849
2023-07-25  1.6176  1.6306  1.6053
2023-07-26  1.6337  1.6176  1.6306
2023-07-27  1.6104  1.6337  1.6176
2023-07-28  1.5839  1.6104  1.6337
2023-07-31  1.6537  1.5839  1.6104
2023-08-01  1.6458  1.6537  1.5839
2023-08-02  1.6618  1.6458  1.6537
2023-08-03  1.6340  1.6618  1.6458
2023-08-04  1.6167  1.6340  1.6618


In [141]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
import datetime

x=db[["prima1","prima2"]]
y=db["target"]
print(x.head())
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.7,random_state=0)
model=ARIMA(ytrain,order=(1,0,1))
modelfit=model.fit()
# start=(db.index[-1]+datetime.timedelta(days=4))
# end=(start+datetime.timedelta(days=4))
# start=start.date()
# end=end.date()
predictions = modelfit.predict(start=len(ytrain),end=len(ytrain)+len(ytest),typ="levels")
pre=pd.DataFrame({"value":predictions})
# pre["value"]=scalertarget.inverse_transform(pre[["value"]])
# print(predictions)
print(pre)
print(db.tail())



            prima1  prima2
date                      
1973-08-22  0.6710  0.6735
1973-08-23  0.6580  0.6710
1973-08-24  0.6675  0.6580
1973-08-27  0.6660  0.6675
1973-08-28  0.6590  0.6660


c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not mo

          value
8778   1.293370
8779   1.293370
8780   1.293370
8781   1.293370
8782   1.293370
...         ...
12536  1.293323
12537  1.293323
12538  1.293323
12539  1.293323
12540  1.293323

[3763 rows x 1 columns]
            target  prima1  prima2
date                              
2023-07-31  1.6537  1.5839  1.6104
2023-08-01  1.6458  1.6537  1.5839
2023-08-02  1.6618  1.6458  1.6537
2023-08-03  1.6340  1.6618  1.6458
2023-08-04  1.6167  1.6340  1.6618


c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


In [135]:
# x=db[["prima1","prima2"]]
# y=db["target"]
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics import r2_score
# model2=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
# model2.fit(xtrain,ytrain)
# new=pd.DataFrame({"prima1":[0.702022],
#                   "prima2":[0.666730]},index=[0])

# pred2=model2.predict(new)
# newpred=pd.DataFrame({"value":pred2})
# newpred["value"]=scalertarget.inverse_transform(newpred[["value"]])
# print(newpred)
